In [1]:
!pip install transformers
!pip install torch


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
import re

In [3]:
file_path = 'commentary_2023.csv'

df = pd.read_csv(file_path)

print(df.head())

print(df.shape)

   Unnamed: 0                                         commentary
0          19  Travis Head | Player of the Match - 137(120): ...
1          20  Adam Zampa: Interesting to see the strategy to...
2          21  Mitchell Starc: He (Cummins) was phenomenal, h...
3          22  Steve Smith: Incredible feeling! Atmosphere wa...
4          23  Mitch Marsh: Pure elation right now. Been the ...
(375, 2)


In [4]:
df_cleaned = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
all_text = " ".join(df_cleaned["commentary"].dropna().astype(str))

words = re.findall(r'\b\w+\b', all_text.lower())  # Convert to lowercase for consistent counting

word_counts = Counter(words)

most_common_words = word_counts.most_common(50)

most_used_words = pd.DataFrame(most_common_words, columns=["Word", "Count"])

In [6]:
tokenizer = AutoTokenizer.from_pretrained("sppm/cric-tweets-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("sppm/cric-tweets-sentiment-analysis")

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_label = torch.argmax(predictions).item()  # Get the index of the highest score
    return sentiment_label

In [7]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_label = torch.argmax(predictions).item()  # Get the index of the highest score
    return "positive" if sentiment_label == 1 else "negative"

# Apply sentiment analysis to the 'commentary' column
df_cleaned['sentiment'] = df_cleaned['commentary'].apply(lambda x: predict_sentiment(str(x)) if pd.notnull(x) else None)

In [8]:
df_cleaned.to_csv("commentaries_with_sentiment.csv", index=False)

print(df_cleaned.head())

                                          commentary sentiment
0  Travis Head | Player of the Match - 137(120): ...  positive
1  Adam Zampa: Interesting to see the strategy to...  positive
2  Mitchell Starc: He (Cummins) was phenomenal, h...  positive
3  Steve Smith: Incredible feeling! Atmosphere wa...  positive
4  Mitch Marsh: Pure elation right now. Been the ...  positive
